# CS484 - Programming Assignment 2 

## Part 3

Make sure that tensorboardcolab is installed.

In [1]:
!pip install tensorboardcolab

In [2]:
%tensorflow_version 1.x

# Load the TensorBoard notebook extension.
%load_ext tensorboard

TensorFlow 1.x selected.


Import the necessary **keras** packages. mnist is no longer imported since we are using our own cat dataset.

In [3]:
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from tensorboardcolab import *
import h5py # to read the dataset
import numpy as np

Using TensorFlow backend.


Connect to Google Drive


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Set the below paths according to your drive contents.

In [0]:
train_path = '/content/drive/My Drive/CS484HW2/train_catvnoncat.h5'
test_path = '/content/drive/My Drive/CS484HW2/test_catvnoncat.h5'

Read the data.

In [0]:
train = h5py.File(train_path, 'r')
x_train = np.array(train['train_set_x'])
y_train = np.array(train['train_set_y'])

In [0]:
test = h5py.File(test_path, 'r')
x_test = np.array(test['test_set_x'])
y_test = np.array(test['test_set_y'])

Initialize batch size, input dimensions, number of classes and total epochs. Batch size is changed to 209 since we utilize all of the training samples in each iteration. Image dimensions are changed to be 64x64, and number of classes is obviously set to 2 (one class for cats, another class for non-cats). 

In [0]:
batch_size, img_rows, img_cols, _ = x_train.shape 
num_classes = 2 # cat and non-cat 

Change the number of channels from 1 to 3.

In [0]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 3, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 3, img_rows, img_cols)
    input_shape = (3, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 3)

We do not change anything in the rest of the sample code.

In [10]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (209, 64, 64, 3)
209 train samples
50 test samples


In [0]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [12]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(num_classes, activation='softmax'))






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 30, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 57600)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               7372928   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)              

In [14]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [15]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=12,
                    verbose=1,
                    validation_data=(x_test, y_test)) # add this argument is to write logs
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 209 samples, validate on 50 samples
Epoch 1/12





209/209 [==============================] - 3s 14ms/step - loss: 0.6836 - acc: 0.5646 - val_loss: 10.2628 - val_acc: 0.3400
Epoch 2/12
209/209 [==============================] - 0s 234us/step - loss: 5.0286 - acc: 0.6555 - val_loss: 0.9209 - val_acc: 0.6600
Epoch 3/12
209/209 [==============================] - 0s 200us/step - loss: 1.8534 - acc: 0.3828 - val_loss: 1.7004 - val_acc: 0.3400
Epoch 4/12
209/209 [==============================] - 0s 212us/step - loss: 1.0210 - acc: 0.6555 - val_loss: 0.7442 - val_acc: 0.3400
Epoch 5/12
209/209 [==============================] - 0s 215us/step - loss: 0.6419 - acc: 0.6411 - val_loss: 0.8051 - val_acc: 0.3400
Epoch 6/12
209/209 [==============================] - 0s 211us/step - loss: 0.6195 - acc: 0.6555 - val_loss: 0.8442 - val_acc: 0.3400
Epoch 7/12
209/209 [=============================

Train and log the configuration: **ADA, EPOCH = 30**

In [16]:
tbc = TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
https://e1270010.ngrok.io


In [17]:
history1 = model.fit(x_train, y_train,
                     batch_size=batch_size,
                     epochs=30,
                     verbose=0,
                     validation_data=(x_test, y_test), 
                     callbacks=[TensorBoardColabCallback(tbc)] # add this argument is to write logs
                     ) 
score1 = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score1[0])
print('Test accuracy:', score1[1])




Test loss: 1.0439030146598816
Test accuracy: 0.38


Train and log the configuration: **SGD, LR = 0.001, EPOCH = 30**

In [18]:
tbc = TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
https://e1270010.ngrok.io


In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.001),
              metrics=['accuracy'])

In [20]:
history2 = model.fit(x_train, y_train,
                     batch_size=batch_size,
                     epochs=30,
                     verbose=0,
                     validation_data=(x_test, y_test), 
                     callbacks=[TensorBoardColabCallback(tbc)] # add this argument is to write logs
                     ) 
score2 = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score2[0])
print('Test accuracy:', score2[1])

Test loss: 0.6045453333854676
Test accuracy: 0.7000000071525574


Train and log the configuration: **SGD, LRs = [0.009, 0.01, 0.011, 0.012, 0.013], EPOCH = 50**

In [0]:
lrs = [0.01, 0.0125, 0.015, 0.0175, 0.02]

In [22]:
histories = []
scores = []
tbcs = []

for lr in lrs:

    tbc = TensorBoardColab()

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.SGD(lr=lr),
                  metrics=['accuracy'])
    
    histories.append(
        model.fit(x_train, y_train,
                  batch_size=batch_size,
                  epochs=50,
                  verbose=0,
                  validation_data=(x_test, y_test), 
                  callbacks=[TensorBoardColabCallback(tbc)] # add this argument is to write logs
                  )
        )

    scores.append(model.evaluate(x_test, y_test, verbose=0))
    print('--- SGD (LR = {}, EPOCH = 50) ---'.format(lr))
    print('Test loss:', scores[-1][0])
    print('Test accuracy:', scores[-1][1])
    print()

Wait for 8 seconds...
TensorBoard link:
https://e1270010.ngrok.io
--- SGD (LR = 0.01, EPOCH = 50) ---
Test loss: 0.5073544406890869
Test accuracy: 0.7800000047683716

Wait for 8 seconds...
TensorBoard link:
https://e1270010.ngrok.io
--- SGD (LR = 0.0125, EPOCH = 50) ---
Test loss: 0.5343038725852967
Test accuracy: 0.8599999904632568

Wait for 8 seconds...
TensorBoard link:
http://e1270010.ngrok.io
--- SGD (LR = 0.015, EPOCH = 50) ---
Test loss: 0.4713124775886536
Test accuracy: 0.8600000023841858

Wait for 8 seconds...
TensorBoard link:
https://e1270010.ngrok.io
--- SGD (LR = 0.0175, EPOCH = 50) ---
Test loss: 0.4988833975791931
Test accuracy: 0.8000000071525574

Wait for 8 seconds...
TensorBoard link:
https://e1270010.ngrok.io
--- SGD (LR = 0.02, EPOCH = 50) ---
Test loss: 0.4547306108474731
Test accuracy: 0.8600000023841858

